In [1]:
import cv2
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def show(img):
    output=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(output)

In [3]:
imori=cv2.imread("./imori.jpg")

In [4]:
# Gray scale
def BGR2GRAY(img):
    b = img[:, :, 0].copy()
    g = img[:, :, 1].copy()
    r = img[:, :, 2].copy()

    # Gray scale
    out = 0.2126 * r + 0.7152 * g + 0.0722 * b
    out = out.astype(np.uint8)

    return out

# Otsu Binalization
def otsu_binarization(img, th=128):
    H, W = img.shape
    out = img.copy()

    max_sigma = 0
    max_t = 0

    # determine threshold
    for _t in range(1, 255):
        v0 = out[np.where(out < _t)]
        m0 = np.mean(v0) if len(v0) > 0 else 0.
        w0 = len(v0) / (H * W)
        v1 = out[np.where(out >= _t)]
        m1 = np.mean(v1) if len(v1) > 0 else 0.
        w1 = len(v1) / (H * W)
        sigma = w0 * w1 * ((m0 - m1) ** 2)
        if sigma > max_sigma:
            max_sigma = sigma
            max_t = _t

    # Binarization
    print("threshold >>", max_t)
    th = max_t
    out[out < th] = 0
    out[out >= th] = 255

    return out


# Erosion
def Erode(img, Erode_time=1):
    H, W = img.shape
    out = img.copy()

    # kernel
    MF = np.array(((0, 1, 0),
                (1, 0, 1),
                (0, 1, 0)), dtype=np.int)

    # each erode
    for i in range(Erode_time):
        tmp = np.pad(out, (1, 1), 'edge')
        # erode
        for y in range(1, H+1):
            for x in range(1, W+1):
                if np.sum(MF * tmp[y-1:y+2, x-1:x+2]) < 255*4:
                    out[y-1, x-1] = 0

    return out


# Dilation
def Dilate(img, Dil_time=1):
    H, W = img.shape

    # kernel
    MF = np.array(((0, 1, 0),
                (1, 0, 1),
                (0, 1, 0)), dtype=np.int)

    # each dilate time
    out = img.copy()
    for i in range(Dil_time):
        tmp = np.pad(out, (1, 1), 'edge')
        for y in range(1, H+1):
            for x in range(1, W+1):
                if np.sum(MF * tmp[y-1:y+2, x-1:x+2]) >= 255:
                    out[y-1, x-1] = 255

    return out

In [5]:
in1=otsu_binarization(BGR2GRAY(imori.copy()))
in2=in1.copy()
dilated=Dilate(in1,1)
eroded=Erode(in2,1)
output=np.clip((dilated-eroded).astype(np.uint8),0,255)
cv2.imwrite("ans1.jpg",output)

threshold >> 127


True

In [6]:
def Morphology_Opening(img,times=1):
    dil=Dilate(img,Dil_time=times)
    erode=Erode(dil,Erode_time=times)
    return erode

In [7]:
ans2=otsu_binarization(BGR2GRAY(imori.copy().astype(np.float32)))
opened=Morphology_Opening(ans2.copy(),times=3)
output=np.clip((np.abs(ans2-opened)*255).astype(np.uint8),0,255)
cv2.imwrite("ans2.jpg",output)

threshold >> 127


True

In [8]:
def Morphology_Closing(img,times=1):
    erode=Erode(img,Erode_time=times)
    dil=Dilate(erode,Dil_time=times)
    return dil

In [9]:
ans3=otsu_binarization(BGR2GRAY(imori.copy().astype(np.float32)))
closed=Morphology_Closing(ans3.copy(),times=3)
output=np.abs(closed-ans3) * 255
cv2.imwrite("ans3.jpg",output)

threshold >> 127


True

In [10]:
def SSD_matching(img,template):
    H,W,C=img.shape
    temp_H,temp_W,_=template.shape
    
    yout,xout=0,0
    inf=1e10
    for y in range(H-temp_H+1):
        for x in range(W-temp_W+1):
            S=np.sum((img[y:y+temp_H,x:x+temp_W]-template)**2)
            if S<inf:
                inf=S
                yout,xout=y,x
    
    output=cv2.rectangle(img,(xout,yout),(xout+temp_W-1,yout+temp_H-1),(0,0,255),1)
    return output

In [11]:
ans4=imori.copy().astype(np.float32)
template=cv2.imread("./imori_part.jpg")
ans4=SSD_matching(ans4,template).astype(np.uint8)
cv2.imwrite("ans4.jpg",ans4)

True

In [12]:
def SAD_matching(img,template):
    H,W,C=img.shape
    temp_H,temp_W,_=template.shape
    
    yout,xout=0,0
    inf=1e10
    for y in range(H-temp_H+1):
        for x in range(W-temp_W+1):
            S=np.sum(np.abs(img[y:y+temp_H,x:x+temp_W]-template))
            if S<inf:
                inf=S
                yout,xout=y,x
    
    output=cv2.rectangle(img,(xout,yout),(xout+temp_W-1,yout+temp_H-1),(0,0,255),1)
    return output

In [13]:
ans5=imori.copy().astype(np.float32)
template=cv2.imread("./imori_part.jpg")
ans5=SAD_matching(ans5,template).astype(np.uint8)
cv2.imwrite("ans5.jpg",ans5)

True

In [14]:
def NCC_matching(img,template):
    H,W,C=img.shape
    temp_H,temp_W,_=template.shape
    
    yout,xout=0,0
    inf=-1e10
    for y in range(H-temp_H+1):
        for x in range(W-temp_W+1):
            upper=np.sum(img[y:y+temp_H,x:x+temp_W]*template)
            lower=np.sqrt(np.sum(img[y:y+temp_H,x:x+temp_W]**2))*np.sqrt(np.sum(template*template))
            S=upper/lower
            if S>inf:
                inf=S
                yout,xout=y,x
    
    output=cv2.rectangle(img,(xout,yout),(xout+temp_W-1,yout+temp_H-1),(0,0,255),1)
    return output

In [15]:
ans6=imori.copy().astype(np.float32)
template=cv2.imread("./imori_part.jpg").astype(np.float32)
ans6=NCC_matching(ans6,template).astype(np.uint8)
cv2.imwrite("ans6.jpg",ans6)

True

In [16]:
def ZNCC_matching(img,template):
    H,W,C=img.shape
    temp_H,temp_W,_=template.shape
    
    mt=np.mean(template,axis=(0,1))
    MT=np.tile(mt,temp_H*temp_W).reshape(temp_H,temp_W,-1)
    
    yout,xout=0,0
    inf=-1e10
    for y in range(H-temp_H+1):
        for x in range(W-temp_W+1):
            I=img[y:y+temp_H,x:x+temp_W]
            mi=np.mean(I,axis=(0,1))
            MI=np.tile(mi,temp_H*temp_W).reshape(temp_H,temp_W,-1)
            T=template
            upper=np.sum((I-MI)*(T-MT))
            lower=np.sqrt(np.sum((I-MI)**2))*np.sqrt(np.sum((T-MT)**2))
            S=upper.astype(np.float32)/lower
            if S>inf:
                inf=S
                yout,xout=y,x
    
    output=cv2.rectangle(img,(xout,yout),(xout+temp_W-1,yout+temp_H-1),(0,0,255),1)
    return output

In [17]:
ans7=imori.copy().astype(np.float32)
template=cv2.imread("./imori_part.jpg").astype(np.float32)
ans7=ZNCC_matching(ans7,template).astype(np.uint8)
cv2.imwrite("ans7.jpg",ans7)

True

In [143]:
def labeling_4nn(img):
    H,W,C=img.shape
    tmp=img.copy()
    img=np.zeros((H+2,W+2,C),dtype=np.uint8)
    img[1:H+1,1:1+W]=tmp
    
    lookuptable={}
    last=0
    for y in range(1,H+1):
        for x in range(1,W+1):
            if img[y,x,0]==0:
                continue
            
            if img[y-1,x,0]==0 and img[y,x-1,0]==0:
                last+=1
                lookuptable[(y,x)]=last
                continue
            
            if img[y-1,x,0]==0:
                lookuptable[(y,x)]=lookuptable[(y,x-1)]
                continue
            if img[y,x-1,0]==0:
                lookuptable[(y,x)]=lookuptable[(y-1,x)]
                continue
            
            if lookuptable[(y-1,x)]<lookuptable[(y,x-1)]:
                label=lookuptable[(y-1,x)]
                xtmp=x
                while xtmp>0 and img[y,xtmp,0]!=0:
                    lookuptable[(y,xtmp)]=label
                    xtmp-=1
            else:
                label=lookuptable[(y,x-1)]
                ytmp=y
                while ytmp>0 and img[ytmp,x,0]!=0:
                    lookuptable[(ytmp,x)]=label
                    ytmp-=1
                
    step=int((256*(256+127)*256+127)/(last+1))
    colormap={}
    
    for i in range(1,last+1):
        num=step*i
        color=[num%256,(num//256)%256,num//256//256%256]
        print(color)
        colormap[i]=color
    for k,v in lookuptable.items():
        img[k[0],k[1]]=colormap[v]
    
    return img[1:1+H,1:1+W].astype(np.uint8)

In [144]:
seg=cv2.imread("seg.png")
ans8=labeling_4nn(seg)
cv2.imwrite("ans8.jpg",ans8)

[15, 224, 47]
[30, 192, 95]
[45, 160, 143]
[60, 128, 191]
[75, 96, 239]
[90, 64, 31]
[105, 32, 79]


True

In [156]:
import queue

def breadth_first_search(now_point,tile,number):
    q=queue.Queue()
    search_dir=[(0,-1),(1,-1),(1,0),(1,1),(0,1)]
    
    q.put(now_point)
    while not q.empty():
        p=q.get()
        for d in search_dir:
            y,x=p[0]+d[0],p[1]+d[1]
            if tile[y,x]==0 or tile[y,x]==number:
                continue
            tile[y,x]=number
            q.put((y,x))
    
    return tile

def labeling_8nn(img):
    H,W,C=img.shape
    
    label=np.zeros((H+2,W+2),dtype=np.int)
    label[1:1+H,1:1+W]=img[...,0]
    output=np.zeros((H+2,W+2,C),dtype=np.uint8)
    
    n=1
    for y in range(1,H+1):
        for x in range(1,W+1):
            if label[y,x]==255:
                label=breadth_first_search((y,x),label,n)
                n+=1
                
    step=int((256*(256+127)*256+127)/(n+1))
    colormap={}
    colormap[0]=[0,0,0]
    
    for i in range(1,n+1):
        num=step*i
        color=[num%256,(num//256)%256,num//256//256%256]
        colormap[i]=color
    for y in range(1,H+1):
        for x in range(1,W+1):
            output[y,x]=colormap[label[y,x]]
    
    return output[1:1+H,1:1+W].astype(np.uint8)
    

In [157]:
seg=cv2.imread("seg.png")
ans9=labeling_8nn(seg)
cv2.imwrite("ans9.jpg",ans9)

True

In [158]:
def alpha_blend(im1,im2,alpha=0.5):
    output=im1*alpha+im2*(1.-alpha)
    
    return output

In [159]:
im1=cv2.imread("imori.jpg").astype(np.float)
im2=cv2.imread("thorino.jpg").astype(np.float)
ans10=alpha_blend(im1,im2,0.6).astype(np.uint8)
cv2.imwrite("ans10.jpg",ans10)

True